# Abstract Matrices

In [1]:
import finite_algebras as alg
import numpy as np
from abstract_matrix import AbstractMatrix

import itertools as it

import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

ex = alg.Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  17 example algebras are available.
  Use "Examples[INDEX]" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-groups.m

## Abstract Matrix over a Finite Field

First, create a finite field, in this case, the [field with 4 elements from Wikipedia](https://en.wikipedia.org/wiki/Finite_field#Field_with_four_elements).

In [2]:
f2 = ex[16]  # No. 16 in the list above
f2.about()


** Field **
Name: F2
Instance ID: 4390934032
Description: Field with 2 elements from paper: 236w06fields.pdf
Order: 2
Identity: 0
Commutative? Yes
Cyclic?: Yes
  Generators: ['1']
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Zero Divisors: None
Multiplicative Cayley Table (showing indices):
[[0, 0], [0, 1]]


In [3]:
f4 = ex[9]  # No. 9 in the list above
f4.about()


** Field **
Name: F4
Instance ID: 4381235024
Description: Field with 4 elements (from Wikipedia)
Order: 4
Identity: 0
Commutative? Yes
Cyclic?: Yes
  Generators: ['1+a', 'a']
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
      2       a       a       2
      3     1+a     1+a       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Zero Divisors: None
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]


The class method, <i>random</i>, will create an Abstract Matrix, of a specified <i>shape</i>, with elements that are randomly chosen from a specified ring or field. (Fixed seed ONLY used to make the random call, below, repeatable.)

In [4]:
def random_matrix_test(shape, algebra, seed=None):
    if seed:
        np.random.seed(seed)
    mat = AbstractMatrix.random(shape, algebra)
    det, inv = print_matrix_info(mat)
    #print(f"\nRandom Matrix over {algebra.name}:\n{mat}")
    return mat, det, inv

def print_matrix_info(mat):
    print("="*20)
    print(f"Matrix over {mat.algebra.name}:\n{mat}")
    det = mat.determinant()
    print(f"\nDeterminant = {det}")
    print(f"\nCofactor Matrix:\n{mat.cofactor_matrix()}")
    inv = mat.inverse()
    print(f"\nInverse:\n{inv}")
    print(f"\nMatrix * Inverse:\n{mat * mat.inverse()}")
    print(f"\nInverse * Matrix:\n{mat.inverse() * mat}\n")
    print("-"*5)
    return det, inv

In [5]:
m1, d1, i1 = random_matrix_test((3, 3), f4, seed=1)

Matrix over F4:

Determinant = 1

Cofactor Matrix:

Inverse:

Matrix * Inverse:

Inverse * Matrix:

-----


In [6]:
m0, d0, i0 = random_matrix_test((3, 3), f4, seed=2)

Matrix over F4:

Determinant = 0

Cofactor Matrix:

Inverse:

Matrix * Inverse:

Inverse * Matrix:

-----


In [7]:
md, dd, id = random_matrix_test((3, 3), f4, seed=3)

Matrix over F4:

Determinant = 1+a

Cofactor Matrix:

Inverse:

Matrix * Inverse:

Inverse * Matrix:

-----


In [8]:
mc, dc, ic = random_matrix_test((3, 3), f4, seed=4)

Matrix over F4:

Determinant = a

Cofactor Matrix:

Inverse:

Matrix * Inverse:

Inverse * Matrix:

-----


## Invertible 2x2 Matrices over a Finite Field

### Generate all Possible NxN Matrices over an Algebra

In [9]:
def generate_all_NxN_matrices_over_algebra(algebra, N):
    """Generate list of all possible NxN matrices using elements from 'algebra'.
    """
    Nsqr = N * N
    combos = list(it.combinations_with_replacement(algebra.elements, Nsqr))
    perms = [list(set(it.permutations(combo))) for combo in combos]
    arrays = [AbstractMatrix(np.array((item)).reshape(N, N), algebra)
              for p in perms for item in p]
    return arrays

def name_mapping(matrices, prefix = 'a', id_element = None, id_name = 'e'):
    n = len(matrices)
    nfill = len(str(n - 1))  # Number of zeros to left-fill integers in element names
    names = [prefix + str(i).zfill(nfill) for i in range(n)]
    if id_element:
        names[matrices.index(id_element)] = id_name
    return dict(zip(names, matrices))

In [10]:
allmats = generate_all_NxN_matrices_over_algebra(f4, 2)
print(len(allmats))
allmats

256


## Create Dict of Matrices

In [ ]:
name_mapping(allmats, prefix = 'A', id_element = AbstractMatrix.identity(2, f4), id_name = '-ID-')

## Closure of Matrices

In [ ]:
def closure_of_matrices(matrices):

    result = matrices

    for pair in it.product(result, result):
        prod = pair[0] * pair[1]
        if prod not in result:
            result.append(prod)

    if len(result) > len(matrices):
        return closure_of_matrices(result)
    else:
        return result

In [ ]:
det1s = [m for m in allmats if m.determinant() == '1']
print(len(det1s))
det1s

## Test Matrices

In [11]:
ps3 = alg.generate_powerset_ring(3)
ps3.about()


** Ring **
Name: PSRing3
Instance ID: 4381288592
Description: Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)
Order: 8
Identity: {}
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
Mult. Identity: {0, 1, 2}
Mult. Commutative? Yes
Zero Divisors: ['{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}']
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1],
 [0, 0, 2, 0, 2, 0, 2, 2],
 [0,

In [12]:
arr0 = np.array([['{1, 2}', '{0, 1, 2}'],
                 ['{0, 2}', '{}']], dtype='<U32')

In [15]:
arr0

array([['{1, 2}', '{0, 1, 2}'],
       ['{0, 2}', '{}']], dtype='<U32')

In [17]:
np.array([['{1, 2}', '{0, 1, 2}'],
       ['{0, 2}', '{}']], dtype='<U32')

array([['{1, 2}', '{0, 1, 2}'],
       ['{0, 2}', '{}']], dtype='<U32')

In [ ]:
mat0 = AbstractMatrix(arr0, ps3)
mat0

In [ ]:
arr0

In [ ]:
print(mat0)

In [ ]:
mat0.array

In [ ]:
mat0

In [ ]:
arr1 = np.array([['{1, 2}', '{0, 1, 2}', '{0, 2}'],
                 ['{0, 2}', '{}', '{1}'],
                 ['{0}', '{1}', '{0, 1}']], dtype='<U32')

mat1 = AbstractMatrix(arr1, ps3)
mat1

In [ ]:
arr2 = [['{0, 1}', '{0, 2}', '{1, 2}'],
        ['{0}'   , '{1, 2}',    '{2}'],
        ['{0, 1}',     '{}',     '{}']]

mat2 = AbstractMatrix(arr2, ps3)
mat2

In [ ]:
arr3 = np.array([['{}', '{0, 1, 2}', '{0, 1, 2}', '{}'],
                 ['{0, 1}', '{0, 2}', '{1, 2}', '{2}'],
                 ['{0, 2}', '{}', '{}', '{0, 1}'],
                 ['{1}', '{0}', '{0, 2}', '{}']], dtype='<U32')

mat3 = AbstractMatrix(arr3, ps3)
mat3

In [ ]:
arr4 = np.array([['{1, 2}']], dtype='<U32')

mat4 = AbstractMatrix(arr4, ps3)
mat4

In [ ]:
arr5 = np.array([['0', '1', 'a'],
                 ['1', 'a', '1+a'],
                 ['1+a', '0', '1']], dtype='<U32')

mat5 = AbstractMatrix(arr5, f4)
mat5

In [ ]:
arr6 = np.array([['1+a', '0', 'a'],
                 ['1+a', '1+a', '0'],
                 ['1', '1', '0']], dtype='<U32')

mat6 = AbstractMatrix(arr6, f4)
mat6

## Matrix Addition & Subtraction

In [ ]:
mat1 + mat2

In [ ]:
try:
    mat2p3 = mat2 + mat3
    print(mat2p3)
except Exception as exc:
    print(exc)

In [ ]:
mat5 + mat6

In [ ]:
mat2 - mat2

In [ ]:
try:
    mat2p3 = mat2 - mat3
    print(mat2p3)
except Exception as exc:
    print(exc)

## Matrix Multiplication

In [ ]:
mat1 * mat2

In [ ]:
mat2 * mat1

In [ ]:
try:
    mat2x3 = mat2 * mat3
    print(mat2x3)
except Exception as exc:
    print(exc)

In [ ]:
mat5 * mat6

## Abstract Matrix of "Zeros"

In [ ]:
matz = AbstractMatrix.zeros((2, 3), ps3)
matz

In [ ]:
matf = AbstractMatrix.zeros((3, 3), f4)
matf

## Abstract Identity Matrix

In [ ]:
AbstractMatrix.identity(4, ps3)

In [ ]:
mat1

In [ ]:
id3 = AbstractMatrix.identity(3, ps3)

mat1 * id3

In [ ]:
id3 * mat1

In [ ]:
id3f = AbstractMatrix.identity(3, f4)
id3f

In [ ]:
mat5 * id3f - mat5

## Random Abstract Matrix

In [ ]:
AbstractMatrix.random((3, 3), ps3)

## Get/Set Matrix Element Values

In [ ]:
mat2

In [ ]:
mat2[0, 1]

In [ ]:
mat2[2, 2] = '{0, 1, 2}'
mat2

In [ ]:
mat2[2, 2] = '{}'
mat2

## Matrix Minor

In [ ]:
mnr2 = mat2.minor(0,0)
mnr2

In [ ]:
mnr3 = mat5.minor(1, 1)
mnr3

## Abstract Cofactor Matrix

In [ ]:
cof2 = mat2.cofactor_matrix()
cof2

In [ ]:
cof3 = mat5.cofactor_matrix()
cof3

## Abstract Matrix Transpose

In [ ]:
cof2_trans = cof2.transpose()
cof2_trans

In [ ]:
cof3.transpose()

## Abstract Matrix Determinant

In [ ]:
mat2.determinant()

Here's a breaksdown of the basic computations required to get the determinant.

NOTE: Addition & multiplication for the ring, ps3, is commutative, so the order of adds and mults is irrelevant, below.

In [ ]:
minor_det_0 = ps3.sub(ps3.mult('{1, 2}', '{}'), ps3.mult('{2}', '{}'))
minor_det_0

In [ ]:
minor_det_1 = ps3.sub(ps3.mult('{0}', '{}'), ps3.mult('{2}', '{0, 1}'))
minor_det_1

In [ ]:
minor_det_2 = ps3.sub(ps3.mult('{0}', '{}'), ps3.mult('{1, 2}', '{0, 1}'))
minor_det_2

In [ ]:
det = ps3.sub(ps3.add(ps3.mult('{0, 1}', minor_det_0),
                      ps3.mult('{1, 2}', minor_det_2)),
              ps3.mult('{0, 2}', minor_det_1))
det

In [ ]:
mat5.determinant()

## Abstract Matrix Inverse

In [ ]:
mat2_inv = mat2.inverse()
mat2_inv

In [ ]:
mat2 * mat2_inv

In [ ]:
mat2_inv * mat2

In [ ]:
mat5_inv = mat5.inverse()
mat5_inv

In [ ]:
mat5 * mat5_inv

In [ ]:
mat5_inv * mat5